<a href="https://colab.research.google.com/github/cereal-d3v/Veg-Vibe/blob/main/RAG_veg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# Install dependencies as needed:
! pip install kagglehub[pandas-datasets] --quiet
import kagglehub
from kagglehub import KaggleDatasetAdapter


# Download latest version
path = kagglehub.dataset_download("akeshkumarhp/vegan-recipies-dataset")

print("Path to dataset files:", path)

100%|██████████| 31.5k/31.5k [00:00<00:00, 6.75MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/akeshkumarhp/vegan-recipies-dataset/versions/1


In [ ]:
from google.colab import userdata

try:
    kaggle_secret = userdata.get('kaggle.json')
    if kaggle_secret:
        print("KAGGLE_JSON secret found.")
    else:
        print("KAGGLE_JSON secret not found or is empty.")
except Exception as e:
    print(f"An error occurred while accessing KAGGLE_JSON secret: {e}")

An error occurred while accessing KAGGLE_JSON secret: Secret kaggle.json does not exist.


In [22]:
path = '/kaggle/input/vegan-recipes/vegan_recipes.csv'

In [25]:
# Load the latest version
df = pd.read_csv(path)

print("First 5 records:")
df.head()

First 5 records:


,Unnamed: 0,href,title,ingredients,preparation
0,0,https://veganuary.com/recipes/rainbow-rice/,Rainbow Rice,\nIngredients\n\nCarrot ribbons (just use a pe...,\nMethod\n\nCook the rice as instructed on the...
1,1,https://veganuary.com/recipes/mfc-nachos/,Nachos,\nIngredients\n\n400g Meatless Farm Co mince (...,\nPreparation\n\nPreheat the oven to 350ºF\nHe...
2,2,https://veganuary.com/recipes/hazelnut-truffles/,Hazelnut Truffles,\nIngredients\n\n100g hazelnuts\n2 tablespoons...,\nMethod\n\nPreheat the oven to 200c\nPut the ...
3,3,https://veganuary.com/recipes/simple-roasted-r...,Simple Roasted Radish by ChicP,\nIngredients\n\n1 170g tub beetroot and horse...,\nPreparation\nPre heat the oven to 160°C\nCut...
4,4,https://veganuary.com/recipes/baked-apple-char...,Baked Apple Charlotte,\nIngredients\n\n2 tbsp rapeseed oil\n75g pitt...,\nPreparation\n\nServes 9\nYou will need an 8i...


**Content-Based Recommender Skeleton**

In [26]:
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

nltk.download('stopwords')
stop = set(stopwords.words('english'))

def clean_text(txt):
    txt = re.sub(r'\d+', '', str(txt))
    txt = re.sub(r'[^\w\s]', '', txt).lower()
    return ' '.join(w for w in txt.split() if w not in stop)

df['cleaned'] = df['ingredients'].apply(clean_text)

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['cleaned'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

def recommend(title, topn=5):
    idx = indices.get(title)
    if idx is None:
        return f"Recipe '{title}' not found."
    sim_scores = sorted(list(enumerate(cosine_sim[idx])), key=lambda x: x[1], reverse=True)
    rec_idxs = [i for i, _ in sim_scores[1:topn+1]]
    return df[['title', 'ingredients']].iloc[rec_idxs]

print(recommend(df['title'].iloc[0]))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


                                               title  \
172                           Chinese Red BBQ Strips   
96                    Low Sugar Sweet and Sour Sauce   
130                                 Tofu Sushi Rolls   
411  Deep Fried Tofu Balls with a Lemon Tahini Sauce   
230                     Korean Crunchy Fillet Burger   

                                           ingredients  
172  \nIngredients\n\nFor the seitan strips:\n– 125...  
96   \nIngredients\n\n1tsp olive oil or coconut oil...  
130  \nIngredients\nSushi rice – 150g\nTofu – 300g\...  
411  \nIngredients\n\n1 Tbsp olive oil\n1 pack firm...  
230  \nIngredients\nOne pack x Quorn Crunchy Fillet...  


**Live Data Ingestion from Vegan Blogs (Scraping)**

In [28]:
import requests
from bs4 import BeautifulSoup

def scrape_vegnews(url="https://vegnews.com/recipes"):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    titles = [h2.text.strip() for h2 in soup.select('h2')]
    links = [a['href'] for a in soup.select('a[href^="/recipes/"]')]
    return list(zip(titles, links))

scrape_vegnews()

[]

**Integrate Docling for RAG (Retrieval-Augmented Generation)**

In [27]:
import requests

def docling_query(question):
    response = requests.post("http://localhost:8000/api/query", json={"question": question})
    return response.json()

print(docling_query("Find vegan pumpkin pie recipe ingredients"))

ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /api/query (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x79cf5af804d0>: Failed to establish a new connection: [Errno 111] Connection refused'))